<a href="https://colab.research.google.com/github/jorgemunozl/vla-test/blob/main/tests/train_val_core.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Core Training Notebook

This notebook contains the essential components for training a PI05 policy:

## What's Included
- **Dataset Loading**: Load and filter episodes from HuggingFace datasets
- **Policy Setup**: Initialize PI05 policy with proper configuration
- **Training Loop**: Core training loop with gradient updates and metrics tracking

## What's Excluded
- Evaluation loops
- WandB logging
- Checkpoint saving strategies
- Model inference/testing

## Usage
1. Set your dataset and policy configuration in the configuration cells
2. Enable `DEBUG_MODE = True` for quick testing with a subset of episodes
3. Set `DEBUG_MODE = False` for full training

## Setup

The following cells set up the environment:
1. Clone the XHUMAN repository
2. Install dependencies
3. Authenticate with HuggingFace Hub
4. Import required libraries

Cloning into 'XHUMAN'...
remote: Enumerating objects: 2078, done.
remote: Counting objects: 100% (284/284), done.
remote: Compressing objects: 100% (179/179), done.
remote: Total 2078 (delta 180), reused 183 (delta 102), pack-reused 1794 (from 2)
Receiving objects: 100% (2078/2078), 7.81 MiB | 18.56 MiB/s, done.
Resolving deltas: 100% (1323/1323), done.


In [2]:
%cd XHUMAN

/content/XHUMAN


In [3]:
!uv pip install -e .[pi]

Using Python 3.12.12 environment at: /usr
Resolved 215 packages in 27.84s
Prepared 46 packages in 24.27s
Uninstalled 15 packages in 1.05s
Installed 46 packages in 222ms
 + async-lru==2.0.5
 + av==15.1.0
 + comm==0.2.3
 + deepdiff==8.6.1
 - diffusers==0.36.0
 + diffusers==0.35.2
 + draccus==0.10.0
 + evdev==1.9.2
 + faker==40.1.2
 + feetech-servo-sdk==1.0.0
 - huggingface-hub==0.36.0
 + huggingface-hub==0.35.3
 + inquirerpy==0.3.4
 - ipykernel==6.17.1
 + ipykernel==7.1.0
 + jedi==0.19.2
 + json5==0.13.0
 + jsonlines==4.0.0
 + jupyter==1.1.1
 - jupyter-client==7.4.9
 + jupyter-client==8.8.0
 + jupyter-lsp==2.3.0
 + jupyterlab==4.5.2
 + jupyterlab-server==2.28.0
 + lerobot==0.4.3 (from git+https://github.com/huggingface/lerobot.git@15724826dd5b18c18656151a02a24ce5fd690c46)
 + mergedeep==1.3.4
 + mypy-extensions==1.1.0
 - notebook==6.5.7
 + notebook==7.5.2
 - nvidia-cudnn-cu12==9.10.2.21
 + nvidia-cudnn-cu12==9.5.1.17
 - nvidia-cusparselt-cu12==0.7.1
 + nvidia-cusparselt-cu12==0.6.3
 - nvi

In [4]:
from huggingface_hub import login
login()

In [5]:
import time
from contextlib import nullcontext
from typing import Any

import torch
from accelerate import Accelerator
from accelerate.utils import DistributedDataParallelKwargs
from torch.optim import Optimizer

from lerobot.configs import parser
from lerobot.datasets.sampler import EpisodeAwareSampler
from lerobot.datasets.utils import cycle
from lerobot.optim.factory import make_optimizer_and_scheduler
from lerobot.policies.pretrained import PreTrainedPolicy
from lerobot.utils.logging_utils import AverageMeter, MetricsTracker
from lerobot.utils.random_utils import set_seed
from lerobot.utils.train_utils import load_training_state
from lerobot.utils.utils import (
    format_big_number,
    has_method,
    init_logging,
)

from xhuman.policies.factory import make_xhuman_policy, make_xhuman_pre_post_processors
from xhuman.configs.train import TrainPipelineConfigXHUMAN
from xhuman.datasets.factory import make_dataset_xhuman
from xhuman.datasets.utils import split_train_eval_episodes
from xhuman.logger import logger

## Helper Functions

These functions handle dataset loading and policy updates. They are designed to work with distributed training using HuggingFace Accelerate.

In [6]:
def load_dataset(cfg: TrainPipelineConfigXHUMAN, episodes: list[int], is_main_process: bool = True, accelerator: Accelerator | None = None):
    """
    Load the dataset for training and evaluation.
    """
    # Dataset loading synchronization: main process downloads first to avoid race conditions
    cfg.dataset.episodes = episodes

    if is_main_process:
        logger.info("Creating dataset")
        dataset = make_dataset_xhuman(cfg)

    accelerator.wait_for_everyone()

    # Now all other processes can safely load the dataset
    if not is_main_process:
        dataset = make_dataset_xhuman(cfg)

    return dataset

In [7]:
def update_policy(
    train_metrics: MetricsTracker,
    policy: PreTrainedPolicy,
    batch: Any,
    optimizer: Optimizer,
    grad_clip_norm: float,
    accelerator: Accelerator,
    lr_scheduler=None,
    lock=None,
) -> tuple[MetricsTracker, dict]:
    """
    Performs a single training step to update the policy's weights.

    This function executes the forward and backward passes, clips gradients, and steps the optimizer and
    learning rate scheduler. Accelerator handles mixed-precision training automatically.

    Args:
        train_metrics: A MetricsTracker instance to record training statistics.
        policy: The policy model to be trained.
        batch: A batch of training data.
        optimizer: The optimizer used to update the policy's parameters.
        grad_clip_norm: The maximum norm for gradient clipping.
        accelerator: The Accelerator instance for distributed training and mixed precision.
        lr_scheduler: An optional learning rate scheduler.
        lock: An optional lock for thread-safe optimizer updates.

    Returns:
        A tuple containing:
        - The updated MetricsTracker with new statistics for this step.
        - A dictionary of outputs from the policy's forward pass, for logging purposes.
    """
    start_time = time.perf_counter()
    policy.train()

    # Let accelerator handle mixed precision
    with accelerator.autocast():
        loss, output_dict = policy.forward(batch)

    # Use accelerator's backward method
    accelerator.backward(loss)

    # Clip gradients if specified
    if grad_clip_norm > 0:
        grad_norm = accelerator.clip_grad_norm_(policy.parameters(), grad_clip_norm)
    else:
        grad_norm = torch.nn.utils.clip_grad_norm_(
            policy.parameters(), float("inf"), error_if_nonfinite=False
        )

    # Optimizer step
    with lock if lock is not None else nullcontext():
        optimizer.step()

    optimizer.zero_grad()

    # Step through pytorch scheduler at every batch instead of epoch
    if lr_scheduler is not None:
        lr_scheduler.step()

    # Update internal buffers if policy has update method
    if has_method(accelerator.unwrap_model(policy, keep_fp32_wrapper=True), "update"):
        accelerator.unwrap_model(policy, keep_fp32_wrapper=True).update()

    train_metrics.loss = loss.item()
    train_metrics.grad_norm = grad_norm.item()
    train_metrics.lr = optimizer.param_groups[0]["lr"]
    train_metrics.update_s = time.perf_counter() - start_time
    return train_metrics, output_dict

In [8]:
from xhuman.policies.pi05.configuration_pi05 import PI05Config

policy_config = PI05Config(repo_id="none",device="cuda")

In [ ]:
# If test PI05 policy
#from lerobot.policies.pi05 import PI05Config

#policy_config = PI05Config(repo_id="none",device="cuda")

## Configuration and Setup

Configure your dataset and policy settings here. The dataset configuration specifies which HuggingFace repository to load, and the policy configuration sets up the PI05 model architecture.

In [9]:
from xhuman.configs.default import LerobotDatasetConfig

dataset_config = LerobotDatasetConfig(
    repo_id="NONHUMAN-RESEARCH/pick-and-place-fruits-v2-test",
)

In [10]:
# Load configuration (you can modify this to load from a config file or set directly)
# Example: cfg = TrainPipelineConfigXHUMAN.from_dict({...})
# For now, we'll use the parser to load from command line or config file
# In notebook, you might want to set cfg directly


cfg = TrainPipelineConfigXHUMAN(
    dataset=dataset_config,
    policy=policy_config # Example policy configuration, replace with your actual policy path
)
cfg.validate()

## Training Setup

Initialize the Accelerator for distributed training and set up the training environment. The accelerator automatically handles:
- Multi-GPU training
- Mixed precision training
- Gradient synchronization across processes

In [11]:
# Create Accelerator
# It will automatically detect if running in distributed mode or single-process mode
# We set step_scheduler_with_optimizer=False to prevent accelerate from adjusting the lr_scheduler steps based on the num_processes
# We set find_unused_parameters=True to handle models with conditional computation
ddp_kwargs = DistributedDataParallelKwargs(find_unused_parameters=True)
accelerator = Accelerator(step_scheduler_with_optimizer=False, kwargs_handlers=[ddp_kwargs])

init_logging(accelerator=accelerator)

# Determine if this is the main process (for logging and checkpointing)
is_main_process = accelerator.is_main_process

# Set seed if specified
if cfg.seed is not None:
    set_seed(cfg.seed, accelerator=accelerator)

# Use accelerator's device
device = accelerator.device
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True

In [12]:
# ============================================================================
# Dataset Loading with Episode Filtering
# ============================================================================
# This cell loads the dataset with proper episode filtering.
# For debugging: Set DEBUG_MODE = True to use only a subset of episodes
# For production: Set DEBUG_MODE = False to use all available episodes

DEBUG_MODE = True  # Set to False for full training
DEBUG_MAX_EPISODES = 3  # Use only first N episodes for debugging

# First, get total episodes count (load minimal dataset to check)
if is_main_process:
    temp_dataset = make_dataset_xhuman(cfg)
    total_episodes = temp_dataset.meta.total_episodes
    del temp_dataset
    logger.info(f"Total episodes available: {total_episodes}")
else:
    # For non-main processes, use a reasonable default
    # In practice, this will be synced after main process loads
    total_episodes = 4  # Fallback - adjust if needed

accelerator.wait_for_everyone()

# Limit episodes for debugging
if DEBUG_MODE:
    episodes = list(range(min(DEBUG_MAX_EPISODES, total_episodes)))
    if is_main_process:
        logger.info(f"DEBUG MODE: Using only {len(episodes)} episodes")
else:
    episodes = list(range(total_episodes))

# Split episodes
train_episodes, eval_episodes = split_train_eval_episodes(
    episodes, split_ratio=cfg.split_ratio, seed=42
)

# Load dataset with ONLY train episodes (proper way to filter)
# This uses the load_dataset helper function which sets cfg.dataset.episodes
if is_main_process:
    logger.info(f"Loading train dataset with {len(train_episodes)} episodes")
dataset = load_dataset(cfg, train_episodes, is_main_process=is_main_process, accelerator=accelerator)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

meta/tasks.parquet:   0%|          | 0.00/2.29k [00:00<?, ?B/s]

meta/episodes/chunk-000/file-000.parquet:   0%|          | 0.00/117k [00:00<?, ?B/s]

info.json: 0.00B [00:00, ?B/s]

...

README.md: 0.00B [00:00, ?B/s]

[04:14:59] INFO     Total episodes available: 11                                     ]8;id=370116;file:///tmp/ipython-input-1033998473.py\ipython-input-1033998473.py]8;;\:]8;id=65997;file:///tmp/ipython-input-1033998473.py#16\16]8;;\

           INFO     DEBUG MODE: Using only 3 episodes                                ]8;id=232594;file:///tmp/ipython-input-1033998473.py\ipython-input-1033998473.py]8;;\:]8;id=703849;file:///tmp/ipython-input-1033998473.py#28\28]8;;\

           INFO     Loading train dataset with 2 episodes                            ]8;id=146316;file:///tmp/ipython-input-1033998473.py\ipython-input-1033998473.py]8;;\:]8;id=772246;file:///tmp/ipython-input-1033998473.py#40\40]8;;\

           INFO     Creating dataset                                                  ]8;id=619176;file:///tmp/ipython-input-1989669429.py\ipython-input-1989669429.py]8;;\:]8;id=442417;file:///tmp/ipython-input-1989669429.py#9\9]8;;\

In [13]:
# Create policy
if is_main_process:
    logger.info("Creating policy")
policy = make_xhuman_policy(
    cfg=cfg.policy,
    ds_meta=dataset.meta,
)

accelerator.wait_for_everyone()

           INFO     Creating policy                                                   ]8;id=631262;file:///tmp/ipython-input-3499831122.py\ipython-input-3499831122.py]8;;\:]8;id=27824;file:///tmp/ipython-input-3499831122.py#3\3]8;;\

processor_config.json:   0%|          | 0.00/253 [00:00<?, ?B/s]

processing_action_tokenizer.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/physical-intelligence/fast:
- processing_action_tokenizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json:   0%|          | 0.00/322 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

## Dataset and Model Information

Display metadata about the loaded dataset and model. This includes:
- Total number of episodes and frames
- Model parameter counts
- Effective batch size (accounting for distributed training)

In [14]:
# Display dataset metadata and model configuration
if is_main_process:
    from pprint import pprint

    print("=" * 80)
    print("DATASET METADATA")
    print("=" * 80)
    print(f"\nDataset Repository: {dataset.repo_id}")
    print(f"Total Episodes: {dataset.meta.total_episodes}")
    print(f"Training Episodes: {len(train_episodes)}")
    print(f"Number of Frames: {dataset.num_frames:,}")
    print(f"Number of Episodes (loaded): {dataset.num_episodes}")


    print("\n" + "=" * 80)

DATASET METADATA

Dataset Repository: NONHUMAN-RESEARCH/pick-and-place-fruits-v2-test
Total Episodes: 11
Training Episodes: 2
Number of Frames: 2,924
Number of Episodes (loaded): 2



In [15]:
processor_kwargs = {}
postprocessor_kwargs = {}

if (cfg.policy.pretrained_path and not cfg.resume) or not cfg.policy.pretrained_path:
    processor_kwargs["dataset_stats"] = dataset.meta.stats

if cfg.policy.pretrained_path is not None:
    processor_kwargs["preprocessor_overrides"] = {
        "device_processor": {"device": device.type},
        "normalizer_processor": {
            "stats": dataset.meta.stats,
            "features": {**policy.config.input_features, **policy.config.output_features},
            "norm_map": policy.config.normalization_mapping,
        },
    }
    postprocessor_kwargs["postprocessor_overrides"] = {
        "unnormalizer_processor": {
            "stats": dataset.meta.stats,
            "features": policy.config.output_features,
            "norm_map": policy.config.normalization_mapping,
        },
    }

In [ ]:
# This cell was removed - autoreload is not needed for production training

In [16]:
# Create processors
preprocessor, postprocessor = make_xhuman_pre_post_processors(
    policy_cfg=cfg.policy,
    pretrained_path=cfg.policy.pretrained_path,
    **processor_kwargs,
    **postprocessor_kwargs,
)

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

In [17]:
# Create optimizer and scheduler
if is_main_process:
    logger.info("Creating optimizer and scheduler")
optimizer, lr_scheduler = make_optimizer_and_scheduler(cfg, policy)

step = 0  # number of policy updates

# Resume from checkpoint if needed
if cfg.resume:
    step, optimizer, lr_scheduler = load_training_state(cfg.checkpoint_path, optimizer, lr_scheduler)

[04:16:28] INFO     Creating optimizer and scheduler                                  ]8;id=439898;file:///tmp/ipython-input-1795964688.py\ipython-input-1795964688.py]8;;\:]8;id=231148;file:///tmp/ipython-input-1795964688.py#3\3]8;;\

In [18]:
# Print training info
if is_main_process:
    num_learnable_params = sum(p.numel() for p in policy.parameters() if p.requires_grad)
    num_total_params = sum(p.numel() for p in policy.parameters())
    logger.info(f"Output dir: {cfg.output_dir}")
    logger.info(f"Steps: {cfg.steps} ({format_big_number(cfg.steps)})")
    logger.info(f"Dataset frames: {dataset.num_frames} ({format_big_number(dataset.num_frames)})")
    logger.info(f"Dataset episodes: {dataset.num_episodes}")
    num_processes = accelerator.num_processes
    effective_bs = cfg.batch_size * num_processes
    logger.info(f"Effective batch size: {cfg.batch_size} x {num_processes} = {effective_bs}")
    logger.info(f"Learnable params: {num_learnable_params} ({format_big_number(num_learnable_params)})")
    logger.info(f"Total params: {num_total_params} ({format_big_number(num_total_params)})")

           INFO     Output dir: outputs/train/2026-01-14/04-14-45_pi05_ki             ]8;id=795667;file:///tmp/ipython-input-4288334519.py\ipython-input-4288334519.py]8;;\:]8;id=844962;file:///tmp/ipython-input-4288334519.py#5\5]8;;\

           INFO     Steps: 100000 (100K)                                              ]8;id=163032;file:///tmp/ipython-input-4288334519.py\ipython-input-4288334519.py]8;;\:]8;id=225772;file:///tmp/ipython-input-4288334519.py#6\6]8;;\

           INFO     Dataset frames: 2924 (3K)                                         ]8;id=101414;file:///tmp/ipython-input-4288334519.py\ipython-input-4288334519.py]8;;\:]8;id=376417;file:///tmp/ipython-input-4288334519.py#7\7]8;;\

           INFO     Dataset episodes: 2                                               ]8;id=45561;file:///tmp/ipython-input-4288334519.py\ipython-input-4288334519.py]8;;\:]8;id=765179;file:///tmp/ipython-input-4288334519.py#8\8]8;;\

           INFO     Effective batch size: 8 x 1 = 8                                  ]8;id=82627;file:///tmp/ipython-input-4288334519.py\ipython-input-4288334519.py]8;;\:]8;id=578856;file:///tmp/ipython-input-4288334519.py#11\11]8;;\

           INFO     Learnable params: 3616757520 (4B)                                ]8;id=903565;file:///tmp/ipython-input-4288334519.py\ipython-input-4288334519.py]8;;\:]8;id=379201;file:///tmp/ipython-input-4288334519.py#12\12]8;;\

           INFO     Total params: 3616757520 (4B)                                    ]8;id=693384;file:///tmp/ipython-input-4288334519.py\ipython-input-4288334519.py]8;;\:]8;id=238968;file:///tmp/ipython-input-4288334519.py#13\13]8;;\

In [19]:
# Create dataloader
if hasattr(cfg.policy, "drop_n_last_frames"):
    logger.info(f"Dropping {cfg.policy.drop_n_last_frames} last frames")
    shuffle = False
    sampler = EpisodeAwareSampler(
        dataset.meta.episodes["dataset_from_index"],
        dataset.meta.episodes["dataset_to_index"],
        drop_n_last_frames=cfg.policy.drop_n_last_frames,
        shuffle=True,
    )
else:
    logger.info("Not dropping any frames")
    shuffle = True
    sampler = None



           INFO     Not dropping any frames                                          ]8;id=105907;file:///tmp/ipython-input-4111286182.py\ipython-input-4111286182.py]8;;\:]8;id=398591;file:///tmp/ipython-input-4111286182.py#12\12]8;;\

HERE

In [ ]:
dataloader = torch.utils.data.DataLoader(
    dataset,
    num_workers=cfg.num_workers,
    batch_size=2,
    shuffle=shuffle and not cfg.dataset.streaming,
    sampler=sampler,
    pin_memory=device.type == "cuda",
    drop_last=False,
    prefetch_factor=2 if cfg.num_workers > 0 else None,
)

In [ ]:
# Prepare everything with accelerator
accelerator.wait_for_everyone()
policy, optimizer, dataloader, lr_scheduler = accelerator.prepare(
    policy, optimizer, dataloader, lr_scheduler
)
dl_iter = cycle(dataloader)

policy.train()

# Setup metrics tracking
train_metrics = {
    "loss": AverageMeter("loss", ":.3f"),
    "grad_norm": AverageMeter("grdn", ":.3f"),
    "lr": AverageMeter("lr", ":0.1e"),
    "update_s": AverageMeter("updt_s", ":.3f"),
    "dataloading_s": AverageMeter("data_s", ":.3f"),
}

effective_batch_size = cfg.batch_size * accelerator.num_processes
train_tracker = MetricsTracker(
    effective_batch_size,
    dataset.num_frames,
    dataset.num_episodes,
    train_metrics,
    initial_step=step,
    accelerator=accelerator,
)

HERE

## Training Loop

The main training loop iterates through batches, performs forward/backward passes, and updates the policy weights.

**Note**: The loop below runs for 6 steps as an example. For full training, replace with:
```python
while step < cfg.steps:
    # ... training code ...
```

Metrics are logged at intervals specified by `cfg.log_freq`.

In [20]:
# Training initialization
# This logs the start of training and shows how many episodes will be used
if is_main_process:
    logger.info("Start offline training on a fixed dataset")
    logger.info(f"Train episodes: {len(train_episodes)}")
    logger.info(f"Total training steps: {cfg.steps}")

           INFO     Start offline training on a fixed dataset                         ]8;id=388162;file:///tmp/ipython-input-2557974599.py\ipython-input-2557974599.py]8;;\:]8;id=372528;file:///tmp/ipython-input-2557974599.py#4\4]8;;\

           INFO     Train episodes: 2                                                 ]8;id=716751;file:///tmp/ipython-input-2557974599.py\ipython-input-2557974599.py]8;;\:]8;id=679514;file:///tmp/ipython-input-2557974599.py#5\5]8;;\

           INFO     Total training steps: 100000                                      ]8;id=764544;file:///tmp/ipython-input-2557974599.py\ipython-input-2557974599.py]8;;\:]8;id=256702;file:///tmp/ipython-input-2557974599.py#6\6]8;;\

In [21]:
from torch.utils.data import Dataset

class SmartSubset(Dataset):
    def __init__(self, dataset, indices):
        self.dataset = dataset
        self.indices = indices

    def __getitem__(self, idx):
        return self.dataset[self.indices[idx]]

    def __len__(self):
        return len(self.indices)

    def __getattr__(self, name):
        # This is the magic part:
        # If the code asks for 'meta', 'fps', etc., and this class doesn't have it,
        # it automatically looks inside the original dataset.
        return getattr(self.dataset, name)

# --- USAGE ---
# Use SmartSubset instead of torch.utils.data.Subset
debug_subset = SmartSubset(dataset, range(0, 50))

# Now create your loader normally
train_dataloader = torch.utils.data.DataLoader(
    debug_subset,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)



In [ ]:
dl_iter = iter(train_dataloader)

In [62]:
train_metrics = {
    "loss": AverageMeter("loss", ":.3f"),
    "grad_norm": AverageMeter("grdn", ":.3f"),
    "lr": AverageMeter("lr", ":0.1e"),
    "update_s": AverageMeter("updt_s", ":.3f"),
    "dataloading_s": AverageMeter("data_s", ":.3f"),
}

effective_batch_size = cfg.batch_size * accelerator.num_processes
train_tracker = MetricsTracker(
    effective_batch_size,
    dataset.num_frames,
    dataset.num_episodes,
    train_metrics,
    initial_step=step,
    accelerator=accelerator,
)

In [63]:
for step in range(0, 2): # Or however many steps you want
    # --- SAFE ITERATOR LOGIC ---
    try:
        batch_ = next(dl_iter)
    except StopIteration:
        # We finished the 50 items! Restart from the beginning.
        dl_iter = iter(train_dataloader)
        batch_ = next(dl_iter)
    start_time = time.perf_counter()
    batch = preprocessor(batch_)

    # print(batch)

    train_tracker, output_dict = update_policy(
        train_tracker,
        policy,
        batch,
        optimizer,
        cfg.optimizer.grad_clip_norm,
        accelerator=accelerator,
        lr_scheduler=lr_scheduler,
    )
    print(train_tracker)
    print(output_dict)

TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [22]:
frames = next(dl_iter)

In [23]:
batch = preprocessor(frames)

In [24]:
batch.keys()

dict_keys(['action', 'next.reward', 'next.done', 'next.truncated', 'info', 'action_is_pad', 'task', 'index', 'task_index', 'episode_index', 'frame_index', 'observation.images.left', 'observation.images.top', 'observation.images.right', 'observation.state', 'observation.language.tokens', 'observation.language.attention_mask'])

In [25]:
batch["action"].shape

torch.Size([2, 50, 14])

In [ ]:
batch["observation.images.left"].shape

torch.Size([2, 3, 376, 672])

# Test loss calculation

In [26]:
tokens_id = "observation.language.tokens"
mask_id = "observation.language.attention_mask"

images, img_masks = policy._preprocess_images(batch)
tokens, masks = batch[tokens_id], batch[mask_id]
noise = None
time = None
actions = policy.prepare_action(batch)

In [68]:
len(images)

3

In [69]:
images[0].shape

torch.Size([2, 3, 224, 224])

In [70]:
images[1].shape

torch.Size([2, 3, 224, 224])

In [ ]:
actions.shape

torch.Size([2, 50, 32])

In [27]:
        tokens = tokens.clone()
        # Embed actions and observations.
        if noise is None:
            noise = policy.model.sample_noise(actions.shape, actions.device)

        if time is None:
            time = policy.model.sample_time(actions.shape[0], actions.device)

        # actions shape: (B, 1, action_dim)
        actions_clone = actions.clone().to("cpu")

        # discrete_actions list of ints: (B, discrete_action_seq_len)
        discrete_actions = policy.model.fast_tokenizer(actions_clone)
        list_discrete_actions = (policy.model.paligemma_with_expert.act_tokens_to_paligemma_tokens(discrete_actions))  # noqa: E501


In [28]:
        list_len_discrete_actions = [
            t.shape[0] for t in list_discrete_actions
        ]

        # Find first padding position (where tokens == 0) for each sample
        # If no padding found, argmax returns 0, so we need to handle that
        pad_positions = (tokens == 0).int().argmax(dim=1)
        # If no padding token found, set to sequence length
        no_pad_mask = (tokens == 0).any(dim=1)
        pad_positions = torch.where(
            no_pad_mask, pad_positions, tokens.shape[1]
        )

        # token_loss_mask shape: (B, seq_len)
        token_loss_mask = torch.zeros_like(
            tokens, dtype=torch.bool, device=tokens.device
        )

        # Clone masks to avoid modifying the original
        masks = masks.clone()

        # Insert discrete action tokens at padding positions and update masks
        for i in range(tokens.shape[0]):
            pad_idx = pad_positions[i].item()
            discrete_len = list_len_discrete_actions[i]
            discrete_tokens = list_discrete_actions[i].to(
                device=tokens.device, dtype=tokens.dtype
            )

            # Check if discrete actions fit in the sequence
            if pad_idx + discrete_len > tokens.shape[1]:
                # Truncate discrete actions if they don't fit
                available_space = tokens.shape[1] - pad_idx
                if available_space > 0:
                    discrete_tokens = discrete_tokens[:available_space]
                    discrete_len = available_space
                else:
                    # No space available, skip this sample's discrete actions
                    continue

            # Insert discrete action tokens at padding position
            tokens[i, pad_idx:pad_idx + discrete_len] = discrete_tokens
            # Mark these positions for loss computation
            token_loss_mask[i, pad_idx:pad_idx + discrete_len] = True
            # Mark discrete action positions as valid (True) in masks
            masks[i, pad_idx:pad_idx + discrete_len] = True

In [55]:
tokens

tensor([[     2,   7071, 235292,   4788,    578,   2040,    573,  16803,    575,
            573,  12220, 235265,   3040, 235292, 235248, 235310, 235274, 235248,
         235276, 235248, 235284, 235308, 235308, 235248, 235274, 235304, 235284,
         235248, 235284, 235274, 235248, 235274, 235310, 235310, 235248, 235276,
         235248, 235284, 235308, 235308,    728, 235274, 235248, 235284, 235308,
         235308, 235248, 235274, 235321, 235318, 235248, 235284, 235308, 235248,
         235274, 235304, 235318,    728, 235274, 235248, 235274, 235284, 235321,
         235248, 235274, 235284, 235321, 235248, 235274, 235284, 235321, 235248,
         235274, 235284, 235321, 235248, 235274, 235284, 235321, 235248, 235274,
         235284, 235321, 235248, 235274, 235284, 235321, 235248, 235274, 235284,
         235321, 235248, 235274, 235284, 235321, 235248, 235274, 235284, 235321,
         235248, 235274, 235284, 235321, 235248, 235274, 235284, 235321, 235248,
         235274, 235284, 235

In [29]:
        prefix_embs, prefix_pad_masks, prefix_att_masks = policy.model.embed_prefix(
            images, img_masks, tokens, masks,
        )
        # Clone prefix_att_masks before modifying since it's a view
        prefix_att_masks = prefix_att_masks.clone()
        # prefix_att_masks structure:
        # [image_embs..., original_lang_tokens, discrete_actions]
        num_image_embs = prefix_embs.shape[1] - tokens.shape[1]


In [64]:
prefix_pad_masks

tensor([[ True,  True,  True,  ..., False, False, False],
        [ True,  True,  True,  ..., False, False, False]], device='cuda:0')

In [66]:
prefix_att_masks

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]], device='cuda:0')

In [30]:
        for i in range(tokens.shape[0]):
            pad_idx = pad_positions[i].item()
            discrete_len = list_len_discrete_actions[i]

            # Check if discrete actions fit
            if pad_idx + discrete_len <= tokens.shape[1]:
                # Calculate position in embedded space
                discrete_action_start_idx = num_image_embs + pad_idx
                discrete_action_end_idx = (
                    discrete_action_start_idx + discrete_len
                )

                # Set att_mask=1 for discrete action tokens (causal masking)
                if discrete_action_end_idx <= prefix_att_masks.shape[1]:
                    prefix_att_masks[
                        i, discrete_action_start_idx:discrete_action_end_idx
                    ] = 1

In [31]:
from xhuman.policies.pi05.modeling_pi_05 import make_att_2d_masks

In [32]:

        time_expanded = time[:, None, None]
        x_t = time_expanded * noise + (1 - time_expanded) * actions
        u_t = noise - actions

        suffix_embs, suffix_pad_masks, suffix_att_masks, adarms_cond = policy.model.embed_suffix(  # noqa: E501
            x_t, time,
        )

        q_proj_dtype = (
            policy.model.paligemma_with_expert.paligemma.language_model.layers[0]
            .self_attn.q_proj.weight.dtype
        )

        if q_proj_dtype == torch.bfloat16:
            suffix_embs = suffix_embs.to(dtype=torch.bfloat16)
            prefix_embs = prefix_embs.to(dtype=torch.bfloat16)

        pad_masks = torch.cat([prefix_pad_masks, suffix_pad_masks], dim=1)
        att_masks = torch.cat([prefix_att_masks, suffix_att_masks], dim=1)

        att_2d_masks = make_att_2d_masks(pad_masks, att_masks)

        suffix_start = prefix_pad_masks.shape[1]

In [44]:
pad_masks

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]], device='cuda:0')

In [45]:
att_masks

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')

In [33]:
        for i in range(tokens.shape[0]):
            pad_idx = pad_positions[i].item()
            discrete_len = list_len_discrete_actions[i]

            # Check if discrete actions fit
            if pad_idx + discrete_len <= tokens.shape[1]:
                # Calculate position in embedded space
                discrete_action_start_idx = num_image_embs + pad_idx
                discrete_action_end_idx = (
                    discrete_action_start_idx + discrete_len
                )

                # Prevent suffix (flow matching) from attending to discrete
                # action tokens
                if discrete_action_end_idx <= att_2d_masks.shape[2]:
                    att_2d_masks[
                        i,
                        suffix_start:,
                        discrete_action_start_idx:discrete_action_end_idx,
                    ] = False

In [34]:

        position_ids = torch.cumsum(pad_masks, dim=1) - 1

        att_2d_masks_4d = policy.model._prepare_attention_masks_4d(att_2d_masks)

In [35]:
import torch.nn.functional as F

In [36]:
att_2d_masks.shape

torch.Size([2, 1018, 1018])

In [57]:
att_2d_masks[0].shape

torch.Size([1018, 1018])

In [59]:
att_2d_masks

tensor([[[ True,  True,  True,  ..., False, False, False],
         [ True,  True,  True,  ..., False, False, False],
         [ True,  True,  True,  ..., False, False, False],
         ...,
         [ True,  True,  True,  ...,  True,  True,  True],
         [ True,  True,  True,  ...,  True,  True,  True],
         [ True,  True,  True,  ...,  True,  True,  True]],

        [[ True,  True,  True,  ..., False, False, False],
         [ True,  True,  True,  ..., False, False, False],
         [ True,  True,  True,  ..., False, False, False],
         ...,
         [ True,  True,  True,  ...,  True,  True,  True],
         [ True,  True,  True,  ...,  True,  True,  True],
         [ True,  True,  True,  ...,  True,  True,  True]]], device='cuda:0')

In [36]:
        def forward_func(prefix_embs,
                         suffix_embs,
                         att_2d_masks_4d,
                         position_ids,
                         adarms_cond,
                         ):
            # Returns both prefix_out and suffix_out for dual loss computation
            (prefix_out, suffix_out), _ = policy.model.paligemma_with_expert.forward(
                attention_mask=att_2d_masks_4d,
                position_ids=position_ids,
                past_key_values=None,
                inputs_embeds=[prefix_embs, suffix_embs],
                use_cache=False,
                adarms_cond=[None, adarms_cond],
            )
            return prefix_out, suffix_out

        prefix_out, suffix_out = policy.model._apply_checkpoint(
            forward_func, prefix_embs, suffix_embs,
            att_2d_masks_4d, position_ids, adarms_cond
        )

        suffix_out = suffix_out[:, -policy.model.config.chunk_size:]
        suffix_out = suffix_out.to(dtype=torch.float32)

In [37]:
        def action_out_proj_func(suffix_out):
            return policy.model.action_out_proj(suffix_out)

        v_t = policy.model._apply_checkpoint(action_out_proj_func, suffix_out)

        # v_t shape: [B, chunk_size, action_dim]
        flow_matching_loss = F.mse_loss(u_t, v_t, reduction="none")
        flow_matching_loss = flow_matching_loss.mean(dim=(1, 2))

        # prefix_out shape: [B, prefix_seq_len, hidden_dim]
        def lm_head_func(prefix_out):
            return policy.model.paligemma_with_expert.paligemma.lm_head(prefix_out)

        # logits shape: [B, prefix_seq_len, vocab_size]
        logits = policy.model._apply_checkpoint(lm_head_func, prefix_out)

        num_image_embs = prefix_embs.shape[1] - tokens.shape[1]
        lang_logits = logits[:, num_image_embs:, :]

        # shift_logits shape: [B, lang_seq_len-1, vocab_size]
        shift_logits = lang_logits[:, :-1, :].contiguous()
        # shift_labels shape: [B, lang_seq_len-1]
        shift_labels = tokens[:, 1:].contiguous()

        # shift_mask shape: [B, lang_seq_len-1]
        shift_mask = token_loss_mask[:, 1:].contiguous()

In [54]:
flat_labels

tensor([  7071, 235292,   4788,    578,   2040,    573,  16803,    575,    573,
         12220, 235265,   3040, 235292, 235248, 235310, 235274, 235248, 235276,
        235248, 235284, 235308, 235308, 235248, 235274, 235304, 235284, 235248,
        235284, 235274, 235248, 235274, 235310, 235310, 235248, 235276, 235248,
        235284, 235308, 235308,    728, 235274, 235248, 235284, 235308, 235308,
        235248, 235274, 235321, 235318, 235248, 235284, 235308, 235248, 235274,
        235304, 235318,    728, 235274, 235248, 235274, 235284, 235321, 235248,
        235274, 235284, 235321, 235248, 235274, 235284, 235321, 235248, 235274,
        235284, 235321, 235248, 235274, 235284, 235321, 235248, 235274, 235284,
        235321, 235248, 235274, 235284, 235321, 235248, 235274, 235284, 235321,
        235248, 235274, 235284, 235321, 235248, 235274, 235284, 235321, 235248,
        235274, 235284, 235321, 235248, 235274, 235284, 235321, 235248, 235274,
        235284, 235321, 235248, 235274, 

In [38]:
        # flat_logits shape: [B*(lang_seq_len-1), vocab_size]
        flat_logits = shift_logits.view(-1, shift_logits.size(-1))
        # flat_labels shape: [B*(lang_seq_len-1)]
        flat_labels = shift_labels.view(-1)
        flat_mask = shift_mask.view(-1)

        # ce_loss_per_token shape: [B*(lang_seq_len-1)]
        ce_loss_per_token = F.cross_entropy(
            flat_logits, flat_labels, reduction='none'
        )

        # masked_ce shape: [B, lang_seq_len-1]
        masked_ce = (ce_loss_per_token * flat_mask).view(tokens.shape[0], -1)  # noqa: E501
        # ce_loss shape: [B]
        ce_loss = masked_ce.sum(dim=1) / (shift_mask.sum(dim=1) + 1e-8)

In [60]:
flat_logits

tensor([[ 0.0000,  0.4362,  2.5858,  ...,  3.3793,  1.0394, -3.0672],
        [ 0.0000, -2.1812,  2.2685,  ..., -0.0185, -0.7638,  2.9919],
        [ 0.0000,  5.3782,  1.8034,  ..., -1.1482, -0.0530,  4.1088],
        ...,
        [ 0.0000,  0.6198,  1.1345,  ..., -3.7817,  1.0893,  0.4086],
        [ 0.0000,  0.6487, -0.0984,  ..., -2.5575,  1.2583,  0.5814],
        [ 0.0000, -0.1911, -1.6306,  ..., -0.7462,  1.3804,  0.4861]],
       device='cuda:0', grad_fn=<ViewBackward0>)

In [41]:
flat_mask

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, 

In [39]:

ce_loss, flow_matching_loss

(tensor([14.0854, 14.1881], device='cuda:0', grad_fn=<DivBackward0>),
 tensor([1.6111, 1.6656], device='cuda:0', grad_fn=<MeanBackward1>))

The follow code runs

In [46]:
dl_iter = iter(train_dataloader)

In [52]:
import time

In [53]:
for step in range(0, 8): # Or however many steps you want
    # --- SAFE ITERATOR LOGIC ---
    try:
        batch_ = next(dl_iter)
    except StopIteration:
        # We finished the 50 items! Restart from the beginning.
        dl_iter = iter(train_dataloader)
        batch_ = next(dl_iter)
    batch = preprocessor(batch_)

    # print(batch)

    train_tracker, output_dict = update_policy(
        MetricsTracker,
        policy,
        batch,
        optimizer,
        cfg.optimizer.grad_clip_norm,
        accelerator=accelerator,
        lr_scheduler=lr_scheduler,
    )


TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [ ]:
dl_iter = iter(dataloader)

## Training Completion

After the training loop completes, metrics are logged and the accelerator is cleaned up.

**Note**: Any debug cells below this point (if present) can be ignored - they were used during development to test dataset subsetting approaches. The proper way to filter episodes is now handled via `DEBUG_MODE` in the dataset loading cell.

In [40]:
# Main training loop
# Replace the range(0, 6) with while step < cfg.steps: for full training
if is_main_process:
    logger.info("Starting training loop")

while step < cfg.steps:
    # Measure data loading time
    start_time = time.perf_counter()
    batch = next(dl_iter)
    batch = preprocessor(batch)
    train_tracker.dataloading_s = time.perf_counter() - start_time

    # Update policy (forward pass, backward pass, optimizer step)
    train_tracker, output_dict = update_policy(
        train_tracker,
        policy,
        batch,
        optimizer,
        cfg.optimizer.grad_clip_norm,
        accelerator=accelerator,
        lr_scheduler=lr_scheduler,
    )

    step += 1
    train_tracker.step()

    # Log metrics at specified intervals
    is_log_step = cfg.log_freq > 0 and step % cfg.log_freq == 0 and is_main_process
    if is_log_step:
        logger.info(train_tracker)
        train_tracker.reset_averages()

    # Optional: Add checkpoint saving here
    # if cfg.checkpoint_freq > 0 and step % cfg.checkpoint_freq == 0:
    #     save_checkpoint(...)

if is_main_process:
    logger.info(f"Training completed! Total steps: {step}")

[04:20:11] INFO     Starting training loop                                            ]8;id=721590;file:///tmp/ipython-input-1209333781.py\ipython-input-1209333781.py]8;;\:]8;id=584004;file:///tmp/ipython-input-1209333781.py#4\4]8;;\

AttributeError: 'Tensor' object has no attribute 'perf_counter'

In [ ]:
# ============================================================================
# Cleanup
# ============================================================================
# Synchronize all processes and clean up accelerator resources
accelerator.wait_for_everyone()
accelerator.end_training()

if is_main_process:
    logger.info("Training session ended successfully")